In [ ]:
!unzip /content/drive/My\ Drive/hist/histopathologic-cancer-detection.zip

We are unzipping our dataset into colab using commandline tool called Unzip

In [ ]:
import numpy as np 
import pandas as pd 
from skimage.io import imread 
import os
import shutil

**Pandas** library is used to read and work with .CSV fiels
**os** and **shutil** library is used to work with directories and copying of files


In [ ]:
!mkdir /content/train/0
!mkdir /content/train/1
import shutil as sh #move files
import pandas as pd #read csv
df_train=pd.read_csv("/content/train_labels.csv")
for index, row in df_train.iterrows():
    if row['label']==0:
        a=str(row['id'])+'.tif'
        sh.move(('/content/train/'+ a),'/content/train/0/')
    elif row['label']==1:
        a=str(row['id'])+'.tif'
        sh.move(('/content/train/'+ a),'/content/train/1/')

We are creating two directories called 0 & 1 
then we will read data from .csv file which contains image id and its label
according to the label we will move the file from train directory to its respective label directory

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import cv2
from keras import layers
import tensorflow
from keras.layers import Flatten, Dense, Input, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.preprocessing import sequence, image
from keras import layers
from keras.optimizers import Adam
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback
from keras.models import Sequential
import keras


TensorFlow 1.x selected.


Using TensorFlow backend.


Here we are specifying the version of Tensorflow to 1.x
tensorflow 1.x is used because Keras is not compatible with the 2.x version of tensorflow
all other modules from keras are imported to build a model
 

In [ ]:
IMAGE_SIZE = 96

batch_size=512

Default Image size is 96 
Batch size is set to 512

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             rotation_range=90,
                                             zoom_range=0.2, 
                                             width_shift_range=0.1,
                                             height_shift_range=0.1,
                                             shear_range=0.05,
                                             channel_shift_range=0.1,
                                             validation_split=0.2
                             )

object of Image datagenerator is created and it is  used to preprocess images before passing it to neural network model
And the preprocessing parameters were defined 
For more info : https://keras.io/api/preprocessing/image/

In [ ]:
train_generator = datagen.flow_from_directory(
        '/content/train',  # this is the target directory
        color_mode="rgb",
        target_size=(96,96),  # all images will be resized to 150x150
        batch_size=batch_size,
        shuffle=True,
        class_mode='binary',
        seed=45,
        classes=['0','1'],
	      subset="training"       
       )


Found 176021 images belonging to 2 classes.


Train generator is created. This is an iterator which is used to iterate over a directory of images
Random seed is set to 45
color mode is set to rgb


In [ ]:
validation_generator = datagen.flow_from_directory(
        '/content/train',  # this is the target directory
        color_mode="rgb",
        target_size=(96,96),  # all images will be resized to 150x150
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary',
        classes=['0','1'],
        subset="validation"       
       )

Found 44004 images belonging to 2 classes.


validation generator is created. This is an iterator which is used to iterate over a directory of images
Random seed is set to 45
color mode is set to rgb

In [ ]:
model= keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(96,96,3),pooling='avg')
x = model.output



x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)


x = Dense(16)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)


x = Dense(8)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)

x = Dense(4)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)          
x = Dropout(0.25)(x)

predictions = Dense(1, activation='sigmoid')(x)

model= Model(input = model.input, output = predictions)






58892288/58889256 [==============================] - 2s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


VGG16 model is imported and used here
Pretrained weights are Imagenet 
Avg pooling layers are used
Dense/Fully connected network is defined


In [ ]:
filepath="vgg16_2_model.hdf5" # checkpoint
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='max')
adam = keras.optimizers.Adam(lr=0.0001)


here a checkpoint is defined which will save the model after every epoch only if the accuracy is highest 

In [ ]:
model.compile(
          optimizer=adam,
          loss='binary_crossentropy',
          metrics=['binary_accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


The model is compiled here with the adam optimizer and binary cross entropy as loss function


In [ ]:
final_model=model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples //batch_size ,
                    validation_data = validation_generator,
                    validation_steps=validation_generator.samples // batch_size,
                    epochs=50,
                    verbose=1,
                    shuffle=True,
                    callbacks=[checkpoint],
                    max_queue_size=60, 
                    workers=5, 
                    use_multiprocessing=True)




Epoch 1/50
343/343 [==============================] - 604s 2s/step - loss: 0.5516 - binary_accuracy: 0.7261 - val_loss: 0.4716 - val_binary_accuracy: 0.8193

Epoch 00001: val_binary_accuracy improved from -inf to 0.81935, saving model to vgg16_2_model.hdf5
Epoch 2/50
343/343 [==============================] - 582s 2s/step - loss: 0.4705 - binary_accuracy: 0.7985 - val_loss: 0.3909 - val_binary_accuracy: 0.8610

Epoch 00002: val_binary_accuracy improved from 0.81935 to 0.86101, saving model to vgg16_2_model.hdf5
Epoch 3/50
343/343 [==============================] - 578s 2s/step - loss: 0.4324 - binary_accuracy: 0.8263 - val_loss: 0.3256 - val_binary_accuracy: 0.9195

Epoch 00003: val_binary_accuracy improved from 0.86101 to 0.91953, saving model to vgg16_2_model.hdf5
Epoch 4/50
343/343 [==============================] - 583s 2s/step - loss: 0.4001 - binary_accuracy: 0.8452 - val_loss: 0.3186 - val_binary_accuracy: 0.8895

Epoch 00004: val_binary_accuracy did not improve from 0.91953
E

Process ForkPoolWorker-181:
Process ForkPoolWorker-182:
Process ForkPoolWorker-176:
Process ForkPoolWorker-179:
Process ForkPoolWorker-177:
Process ForkPoolWorker-178:
Process ForkPoolWorker-180:
Process ForkPoolWorker-185:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs

KeyboardInterrupt: ignored

using model.fit_generator the model was trained 
for more info https://keras.io/api/models/model_training_apis/#fit-method

In [ ]:
!cp /content/vgg16_2_model.hdf5 /content/drive/My\ Drive